In [1]:
import os
import torch
import catalyst

from catalyst.dl import utils

print(f"torch: {torch.__version__}, catalyst: {catalyst.__version__}")

/home/bml/miniconda3/envs/torch/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
alchemy not available, to install alchemy, run `pip install alchemy-catalyst`.


torch: 1.4.0, catalyst: 20.01.3


# Data Augmentations

In [2]:
import albumentations as albu
from albumentations.pytorch import ToTensorV2 as ToTensor

In [3]:
scaling = (0.25, 1.0)
crop_size = (256, 256)

In [4]:
train_tfms = albu.Compose([
  albu.RandomScale(scaling),
  albu.RandomCrop(crop_size[1], crop_size[0]),
  albu.HorizontalFlip(),
  albu.Normalize(),
  ToTensor(),
])

val_tfms = albu.Compose([
  albu.Resize(512, 1024),
  albu.Normalize(),
  ToTensor(),
])

In [5]:
from torch_semantic_segmentation.data import CityScapesDataset

train_ds = CityScapesDataset(
  '/home/bml/datasets/cities-scapes',
  split='train', transforms=train_tfms)
val_ds = CityScapesDataset(
  '/home/bml/datasets/cities-scapes',
  split='val', transforms=val_tfms)

In [6]:
train_loader = torch.utils.data.DataLoader(train_ds, 
  batch_size=20, num_workers=4, drop_last=True, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds,
  batch_size=8, num_workers=4, drop_last=False, shuffle=False)

In [7]:
from torch import nn
from torch_semantic_segmentation.models import ENet

In [8]:
from torch import optim
from catalyst.contrib.nn import DiceLoss, IoULoss
from catalyst.contrib.nn import RAdam, Lookahead

In [9]:
model = ENet(3, 19)

In [10]:
learning_rate = 0.001

In [11]:
base_optimizer = RAdam(model.parameters(), lr=1e-3, weight_decay=5e-4)
optimizer = Lookahead(base_optimizer)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.25, patience=2)

In [12]:
from catalyst.dl import SupervisedRunner

In [13]:
runner = SupervisedRunner()

In [14]:
from catalyst.dl.callbacks import DiceCallback, IouCallback, \
  CriterionCallback, CriterionAggregatorCallback, ClasswiseIouCallback, AccuracyCallback

In [15]:
from collections import OrderedDict

loaders = OrderedDict([
  ('train', train_loader),
  ('valid', val_loader),
])

In [16]:
runner.train(
  model=model, 
  criterion=nn.CrossEntropyLoss(ignore_index=255),
  optimizer=optimizer, 
  scheduler=scheduler,
  loaders=loaders,
  num_epochs=3,
  callbacks=[
    AccuracyCallback(num_classes=2),
  ],
  verbose=True,
  logdir='./logs',
)

1/3 * Epoch (train):   0% 0/148 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (20) must match the size of tensor b (19) at non-singleton dimension 1

In [ ]:
import gc; gc.collect()